### I) XỬ LÝ DỮ LIỆU TRẬN ĐẤU ĐƯỢC RÚT TRÍCH TỪ API VÀ LƯU TRỮ VÀO DATAFRAME

In [3]:
import requests
import json
import pandas as pd
import os 
import glob
import numpy as np
import csv

#### hàm get_all_json đọc tất cả các file json của từng người chơi và lưu vào biến all_players

path_to_json: lấy đường dẫn của từng người chơi trong từng server

json_files: lấy ra 20 file json của người chơi

vòng for để lưu từng file json vào biến data dưới dạng mảng

In [5]:
def get_all_json(player, region):
    data = []
    path_to_json = 'PROJECT/Data_Match/' + region + '/'+player+'/'
    path_to_json = path_to_json.to_string()
    path_to_json = path_to_json[path_to_json.find(".."):len(path_to_json)]

    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    for i in range(len(json_files)):
        f = open(path_to_json + json_files[i], "r")
        data.append(json.loads(f.read()))
        f.close()
    return data

hàm apply để chạy duyệt từng tên trong 2 server NA và KR

In [13]:
df = pd.read_csv('../PROJECT/Data/data_summoner_tft_csv.csv')
kr = df['kr'][:100]
na = df['na'][:100]

kr_temp = kr.to_frame()
na_temp = na.to_frame()
all_players_kr = kr_temp.apply(get_all_json, args = ("KR",), axis = 1)
all_players_na = na_temp.apply(get_all_json, args = ("NA1",), axis = 1)


FileNotFoundError: [Errno 2] No such file or directory: 'PROJECT/Data/data_summoner_tft_csv.csv'

In [ ]:
all_players = (all_players_kr.append(all_players_na, ignore_index= True))

#### lấy lõi nâng cấp của từng người chơi trong 1 game đó

##### gồm 1 mảng 3 chiều:

    +Chiều thứ nhất: số lượng trận đấu của từng người chơi(20)

    +Chiều thứ hai: sô lượng lõi nâng cấp của 8 người chơi
    
    +Chiều thứ ba: lõi nâng cấp của người chơi

In [ ]:
augments = []
cc = []
augment = []
for player in range(len(all_players)):
    for match in range(len(all_players[0])):
        for each_player in range(8):
            augment.append(all_players[player][match]['info']['participants'][each_player]['augments'])
        
        temp = augment.copy()
        cc.append(temp)
        augment.clear()
        
    test = cc.copy()
    augments.append(test)
    cc.clear()

#### lấy puuid và trận đấu của 8 người chơi

##### puuid là 1 mảng 3 chiều:

    +Chiều thứ nhất: số lượng trận đấu của từng người chơi(20)
    
    +Chiều thứ hai: số lượng người chơi trong 1 trận đấu(thường là 8)

    +Chiều thứ ba: Puuid của từng người chơi
##### id là 1 mảng 2 chiều:

    +Chiều thứ nhất: số lượng trận đấu của người chơi(20)

    +Chiều thứ hai: mã trận đấu(20)

In [ ]:
participants = []
id_match = []
participant = []
id = []
for player in range(len(all_players)):
    for match in range(len(all_players[0])):
        participant.append(all_players[player][match]["metadata"]["participants"])
        id.append(all_players[player][match]["metadata"]["match_id"])   

    temp1 = participant.copy()
    temp2 = id.copy()
    participants.append(temp1)
    id_match.append(temp2)
    participant.clear()
    id.clear()

#### lấy số lượng tộc hệ kích hoạt được(list_traits)

traits name: tên của hệ được kích hoạt

traits num_units: số tướng có trong hệ đó

tier_current: số mốc kích hoạt được trong hệ đó

1 mảng 3 chiều:

    +Chiều thứ nhất: số trận đấu của từng người chơi(20)

    +Chiều thứ hai: số hệ có trong đội hình người chơi

    +Chiều thứ ba: tên hệ, số lượng và cấp của hệ đó

In [ ]:
list_traits = []
in_match = []
in_player = []
for player in range(len(all_players)):
    for match in range(len(all_players[0])):
        for each_player in range(8):
            traits_each_player = all_players[player][match]['info']['participants'][each_player]['traits']
            temp = []
            for trait in range(len(traits_each_player)):
                temp.append(all_players[player][match]['info']['participants'][each_player]['traits'][trait]['name'])
                temp.append(all_players[player][match]['info']['participants'][each_player]['traits'][trait]['num_units'])
                temp.append(all_players[player][match]['info']['participants'][each_player]['traits'][trait]['tier_current'])
            
            test1 = temp.copy()
            in_player.append(test1)
            temp.clear()
        
        test2 = in_player.copy()
        in_match.append(test2)
        in_player.clear()
    
    test3 = in_match.copy()
    list_traits.append(test3)
    in_match.clear()


#### danh sách các tướng có trong đội hình của từng người chơi

character_id: tên của tướng trnog đội hình người chơi

len(items): số lượng đồ mà con tướng đó sử dụng

##### 1 mảng 3 chiều:

    +Chiều thứ nhất: số lượng trận đấu của từng người chơi(20)

    +Chiều thứ hai: số tướng có trong đội hình người chơi

    +Chiều thứ ba: gồm tướng, số lượng đồ của tướng và bậc sao của tướng(1, 2, 3 sao)

In [ ]:
list_champions = []

for player in range(len(all_players)):
    for match in range(len(all_players[0])):
        for each_player in range(8):
            temp = []
            player_champions = all_players[player][match]['info']['participants'][each_player]['units']
            for each_champion in range(len(player_champions)):
                temp.append(all_players[player][match]['info']['participants'][each_player]['units'][each_champion]['character_id'])
            test1 = temp.copy()
            in_player.append(test1)
            temp.clear()
        
        test2 = in_player.copy()
        in_match.append(test2)
        in_player.clear()
    
    test3 = in_match.copy()
    list_champions.append(test3)
    in_match.clear()

In [ ]:
list_items = []

for player in range(len(all_players)):
    for match in range(len(all_players[0])):
        for each_player in range(8):
            temp = []
            player_champions = all_players[player][match]['info']['participants'][each_player]['units']
            for each_champion in range(len(player_champions)):
                temp.append(len(all_players[player][match]['info']['participants'][each_player]['units'][each_champion]['items']))
            test1 = temp.copy()
            in_player.append(test1)
            temp.clear()
        
        test2 = in_player.copy()
        in_match.append(test2)
        in_player.clear()
    
    test3 = in_match.copy()
    list_items.append(test3)
    in_match.clear()

In [ ]:
list_tiers = []

for player in range(len(all_players)):
    for match in range(len(all_players[0])):
        for each_player in range(8):
            temp = []
            player_champions = all_players[player][match]['info']['participants'][each_player]['units']
            for each_champion in range(len(player_champions)):     
                temp.append(all_players[player][match]['info']['participants'][each_player]['units'][each_champion]['tier'])      
            test1 = temp.copy()
            in_player.append(test1)
            temp.clear()
            
        test2 = in_player.copy()
        in_match.append(test2)
        in_player.clear()

    test3 = in_match.copy()
    list_tiers.append(test3)
    in_match.clear()

#### LẤY THỨ HẠNG CỦA TỪNG NGƯỜI CHƠI TRONG MỖI TRẬN ĐẤU

##### gồm 1 mảng 3 chiều: 

    +Chiều thứ nhất: gồm số lượng trận đấu người chơi(20)

    +Chiều thứ hai: gồm số người chơi trong trận

    +Chiều thứ ba: thứ hạng của người chơi

In [ ]:
list_ranks = []
player_rank = []
in_match = []
for player in range(len(all_players)):
    for match in range(len(all_players[0])):
        for each_player in range(8):
            player_rank.append(all_players[player][match]['info']['participants'][each_player]['placement'])
        temp1 = player_rank.copy()
        in_match.append(temp1)
        player_rank.clear()
    temp2 = in_match.copy()
    list_ranks.append(temp2)
    in_match.clear()

### LƯU TẤT CẢ DỮ LIỆU VÀO DATAFRAME

##### tạo 1 dict để lưu trữ từng đội hình của từng trận đấu theo hàng:
    
    +Puuid: Sử dụng hàm flatten để chuyển mảng 2 chiều thành mảng 1 chiều, sau đó ép vào cột đầu của dataframe

    +ID Match: vì 1 trận đấu có 8 người chỉ tương đương với 1 ID Match nên ta sẽ phải lặp lại ID Match 8 lần để phù hợp với kích thước của dataframe

    +

In [ ]:
temp = {'Puuid': np.array(participants).flatten(), 
        'ID Match': np.array(np.array(id_match).repeat(8)).flatten(), 
        'Placement': np.array(list_ranks).flatten(),
        'List Augments': np.array(np.array(augments).flatten()),
        'List Traits': np.array(np.array(list_traits).flatten()),
        'List Champions': np.array(np.array(list_champions).flatten()),
        'List Number Items':np.array(np.array(list_items).flatten()),
        'List Champions Tier':(np.array(np.array(list_tiers).flatten())).flatten()}
process_data_tft_general = pd.DataFrame(temp)

In [ ]:
process_data_tft_general.to_csv("process_data_tft_general_csv.csv")

### II) LẤY ĐỒ CỦA TỪNG TƯỚNG TRONG ĐỘI HÌNH CỦA NGƯỜI CHƠI

#### LẤY ĐỒ CỦA TỪNG TƯỚNG

##### 1 mảng 3 chiều:

    +Chiều thứ nhất: số lượng trận đấu(20)

    +Chiều thứ hai: số người chơi(8)

    +Chiều thứ ba: đồ của từng tướng trong đội hình của người chơi

In [ ]:
list_items = []
in_match = []
in_player = []
for player in range(len(all_players)):
    for match in range(len((all_players[0]))):
        for each_player in range(8):
            items_champion = all_players[player][match]['info']['participants'][each_player]['units']
            temp = []
            for item in range(len(items_champion)):
                temp.append(all_players[player][match]['info']['participants'][each_player]['units'][item]['itemNames'])

            temp1 = temp.copy()
            in_match.append(temp1)
            temp.clear()

        temp2 = in_match.copy()
        in_player.append(temp2)
        in_match.clear()

    temp3 = in_player.copy()
    list_items.append(temp3)
    in_player.clear()  

#### LẤY TÊN CỦA TƯỚNG

##### 1 mảng 3 chiều:

    +Chiều thứ nhất: số trận đấu(20)

    +Chiều thứ hai: số lượng người chơi

    +Chiều thứ ba: 1 list tướng của người chơi đó

In [ ]:
champions = []
in_match = []
in_player = []
for player in range(len(all_players)):
    for match in range(len(all_players[0])):
        for each_player in range(8):
            player_champions = all_players[player][match]['info']['participants'][each_player]['units']
            temp = []
            for name in range(len(player_champions)):
                temp.append(all_players[player][match]['info']['participants'][each_player]['units'][name]['character_id'])

            temp1 = temp.copy()
            in_match.append(temp1)
            temp.clear()

        temp2 = in_match.copy()
        in_player.append(temp2)
        in_match.clear()

    temp3 = in_player.copy()
    champions.append(temp3)
    in_player.clear() 

In [ ]:
temp = {'Puuid': np.array(participants).flatten(), 
        'ID Match': np.array(np.array(id_match).repeat(8)).flatten(), 
        'Champion': np.array(champions).flatten(),
        'List Name Item':np.array(list_items).flatten(),
        'Placement':np.array(list_ranks).flatten()}


process_data_tft_each_item = pd.DataFrame(temp)

In [ ]:
process_data_tft_each_item["List Name Item"]

Series([], Name: List Name Item, dtype: float64)

In [ ]:
process_data_tft_each_item.to_csv("process_data_tft_each_item_csv.csv")